In [13]:
import json
import csv
import os
import glob
import pandas as pd

In [14]:
with open("../visData.json") as f:
    data = json.load(f)

In [19]:
file_names =  []
for item in data:
    file_names.append('..' + item['Highlight Data'])

In [20]:
combined_csv = pd.concat([pd.read_csv(f) for f in file_names ])

combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')

In [52]:
json_data = pd.read_json('visData.json')
#renames and dropped unwanted columns
json_data = json_data.rename(columns = {'article_sha256': 'Article ID'}).drop(columns = ['Highlight Data','Plain Text','Visualization Link','Article Link', 'ID','articleHash'])
json_data.head(3)

,Article ID,Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7d1747829a65cd4dad0faf30a704875a125e9decce3b3b...,"Over 1,000 People Cured of Coronavirus in Italy",Thomas Williams,2020-03-11 18:56:55+00:00
2,be0b18a87d4370fa579180ef26dcb7080598f27f9ec761...,SARS-CoV-2 Can Live on Plastic and Steel for 2...,Kerry Grens,2020-03-12 18:56:55+00:00


In [21]:
combined = pd.read_csv('combined_csv.csv')

In [22]:
combined.head(3)

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN


In [23]:
combined[['Start','End']]

,Start,End
0,-1.0,-1.0
1,-1.0,-1.0
2,-1.0,-1.0
3,-1.0,-1.0
4,-1.0,-1.0
...,...,...
118,-1.0,-1.0
119,-1.0,-1.0
120,-1.0,-1.0
121,3017.0,3197.0


In [24]:
combined['Start,End'] = combined[['Start','End']].apply(tuple,axis = 1)

In [57]:
#merge both csv and json dataframes together
json_csv = pd.merge(left = combined, right = json_data, on = 'Article ID')
json_csv.head()

,Article ID,Credibility Indicator ID,Credibility Indicator Category,Credibility Indicator Name,Points,Indices of Label in Article,Start,End,target_text,"Start,End",Title,Author,Date
0,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E0,Evidence,Alternative explanation,-1.868043,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
1,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,E1,Evidence,Open to evidence,-1.657269,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
2,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H0,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
3,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H1,Holistic,Vaguely Sourced,-1.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00
4,7360da3cdcf83a48e365821654ef0750810f3483efb8e2...,H2,Holistic,Vague Sourcing,-2.000000,[],-1.0,-1.0,NaN,"(-1.0, -1.0)",US might be complementing Iran sanctions with ...,PressTV,2020-03-17 18:56:55+00:00


In [25]:
data = combined.groupby(['Article ID', 'Credibility Indicator Category','Credibility Indicator Name','Start,End']).count()

In [58]:
data

Credibility Indicator ID  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Start,End                                    
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           (-1.0, -1.0)                             1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)                             4   
                                                                                  Vaguely Sourced            (-1.0, -1.0)                             1   
                                                   Language                       Metaphor unhelpful         (387.0, 398.0)                           1   
                                                   Probability                    Acknowledges uncertainty   (259.0, 268.0)                           1   
...                                                                                                                                                 ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Evidence                       Open to evidence           (1805.0, 1815.0)                         1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)                             4   
                                                                                  Vague Sourcing             (-1.0, -1.0)                             1   
                                                   Probability                    Acknowledges uncertainty   (2204.0, 2389.0)                         1   
                                                                                  Open to evidence           (2274.0, 2496.0)                         1   

                                                                                                                               Points  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Start,End                  
3be14d67e2d88964904dcbe7df176bb81dacfc76a6f2e4e... Evidence                       Open to evidence           (-1.0, -1.0)           1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)           4   
                                                                                  Vaguely Sourced            (-1.0, -1.0)           1   
                                                   Language                       Metaphor unhelpful         (387.0, 398.0)         1   
                                                   Probability                    Acknowledges uncertainty   (259.0, 268.0)         1   
...                                                                                                                               ...   
be0b18a87d4370fa579180ef26dcb7080598f27f9ec7618... Evidence                       Open to evidence           (1805.0, 1815.0)       1   
                                                   Holistic                       Qualified Source           (-1.0, -1.0)           4   
                                                                                  Vague Sourcing             (-1.0, -1.0)           1   
                                                   Probability                    Acknowledges uncertainty   (2204.0, 2389.0)       1   
                                                                                  Open to evidence           (2274.0, 2496.0)       1   

                                                                                                                               Indices of Label in Article  \
Article ID                                         Credibility Indicator Category Credibility Indicator Name Start,End                                       
3be14d67e2d88